In [1]:
'''
IMPORTANT!
Requires Python 2
one set up option here:
https://www.dev2qa.com/how-to-run-python-2-code-in-jupyter-notebook/
'''

'\nIMPORTANT!\nRequires Python 2\none set up option here:\nhttps://www.dev2qa.com/how-to-run-python-2-code-in-jupyter-notebook/\n'

# Document Similarity: doc2vec

In [1]:
import pandas as pd
import numpy as np

import re
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.metrics.pairwise import cosine_similarity
import scipy.stats

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mariiabogdanova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
human_evaluation_data = pd.read_csv("Data/AverageSimilarities_fixed.csv")
# human_evaluation_data.head(5)

In [4]:
documents = []
for i in range (0, 50):
    FILENAME = "Data/Docs/{}.txt".format(i)
    with open(FILENAME, 'r') as inputfile:
        lines = inputfile.readlines()
        for line in lines:
            documents.append(line)

In [5]:
documents_df = pd.DataFrame(documents,columns=['documents'])

stopwords_list = stopwords.words('english')
documents_df['documents_cleaned'] = documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stopwords_list) )
documents_df.head(5)

,documents,documents_cleaned
0,The national executive of the strife-torn Demo...,national executive strife torn democrats last ...
1,Cash-strapped financial services group AMP has...,cash strapped financial services group amp she...
2,The United States government has said it wants...,united states government said wants see presid...
3,A radical armed Islamist group with ties to Te...,radical armed islamist group ties tehran baghd...
4,Washington has sharply rebuked Russia over bom...,washington sharply rebuked russia bombings geo...


In [6]:
# Please download models from https://github.com/jhlau/doc2vec
model = gensim.models.doc2vec.Doc2Vec.load('enwiki_dbow/doc2vec.bin')
model2 = gensim.models.doc2vec.Doc2Vec.load('apnews_dbow/doc2vec.bin')

In [7]:
data = []
data2 = []

for doc1, doc2 in zip(human_evaluation_data.Document_1.values, human_evaluation_data.Document_2.values):

    fisrt_text = documents_df.documents_cleaned.values[doc1-1]
    second_text = documents_df.documents_cleaned.values[doc2-1]

    vec1 = model.infer_vector(fisrt_text.split()).reshape(1, -1)
    vec2 = model.infer_vector(second_text.split()).reshape(1, -1)
    
    vec1_2 = model2.infer_vector(fisrt_text.split()).reshape(1, -1)
    vec2_2 = model2.infer_vector(second_text.split()).reshape(1, -1)

    cos_distance = cosine_similarity(vec1, vec2)
    cos_distance2 = cosine_similarity(vec1_2, vec2_2)
    data.append(cos_distance[0][0])    
    data2.append(cos_distance2[0][0])

In [8]:
human_evaluation_data['Similarity_doc2vec_enwiki_dbow'] = data
human_evaluation_data['Similarity_doc2vec_apnews_dbow'] = data2


human_evaluation_data.to_csv('Data/AverageSimilarities_fixed.csv', index=False)

In [9]:
scipy.stats.pearsonr(human_evaluation_data.Similarity_avg.values, data)[0]

0.6257019732333775

In [10]:
scipy.stats.pearsonr(human_evaluation_data.Similarity_avg.values, data2)[0]

0.5573399354085979